# Adamata

## ADAptive Moment estimation cellular automATA

### A cellular automata framework based on the adam optimizer update

The adam update ([Kingma and Ba 2014](https://arxiv.org/abs/1412.6980)) is a popular method for optimizing model parameters based on the first and second moments of parameter gradients with respect to an objective function. 

Those familiar with the adam method (short for adaptive moment estimation) may recognize the equation:

$$
\theta' = \theta + \alpha \frac{m}{\sqrt{v}+\epsilon}
$$

Where $\theta$ and $\theta'$ are the parameters and parameters after updating, respectively, $\alpha$ is a learning rate, $m$ is the first moment, $v$ is the second moment, and $\epsilon$ is a small factor that prevents divsion by $0$. 

To build an adam cellular automata, we can replace the parameters $\theta$ with the cell states at time $t$, $A_t$, and the learning rate with a temporal step size $\Delta t$. 

$$
A_{t+\Delta t} = A_t + \Delta t \frac{m}{\sqrt{v}+\epsilon}
$$

Additionally, the cell state values are squashed or truncated to values between 0 and 1.0, denoted by $[\cdot]_0^1$ below:

$$
A_{t+ \Delta t} = \left[A_t + \Delta t * \frac{m}{\sqrt{v}+\epsilon}\right]_0^1
$$

The first moment will be the exponential average of a growth function $G$ (normally the exponential average of the gradient with respect to loss, in optimization scenarios). 

Taking inspiration from Lenia ([Chan 2019](https://www.complex-systems.com/abstracts/v28_i03_a01/)), I used a Gaussian shifted to output a range from -1.0 to 1.0 as a growth function $G$.

$$
G(x) = 2 e^{-\frac{x-\mu_G}{2\sigma_G^2}} - 1
$$

The growth function takes CA neighborhoods as input, the result of convolving cell states with a convolution kernel $K_n$. To calculate the first moment, we take a running exponential average of the growth function applied to neighborhoods. $\beta_1$ determines the smoothness of the average and the rate of forgetting. 

$$
m_{t+\Delta t} = \beta_1 m_t + (1-\beta_1) G(K_n \circledast A_t)
$$

The second moment is calculated similarly as the running exponential average of the square of the growth function applied to neighborhoods. $\beta_2$ determines the smoothness of the average and the rate of forgetting. 

$$
v_{t+\Delta t} = \beta_2 v_t + (1-\beta_2) G(K_n \circledast A_t)^2
$$

The original adam optimization method includes a bias correction step, which I've omitted from the adam automaton framework. As the first and second moment start from $0.0$, there would be a substantial warmup period for the biased moment estimates, which the bias correction step seeks to correct by division by $1.0$ less the exponential averaging factors raised to the power of the current step, $\beta_1^t$ or $\beta_2^t$. 


I didn't include the bias correction steps in adam CA, but for sake of completeness they are provided below. 

$$
\hat m_{t+1} = \frac{m_{t+1}}{1-\beta_1^t}
$$


$$
\hat v_{t+1} = \frac{v_{t+1}}{1-\beta_2^t}
$$



I started with the same neighborhood function and similar growth parameters to {\itshape Orbium} in Lenia, leading to the rapid discovery of a charismatic glider I call Adorbium (adam + {\itshape Orbium}). Adorbium parameters are: $\beta_1=0.8$, $\beta_2=0.99$, $\epsilon=10^{-8}$, $\mu_G=0.167$, $\sigma_G=0.013$, $\mu_K=0.5$, $\sigma_K=0.15$, with a default kernel diameter of 27 pixels. 

<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/adorbium_4.gif"> -->

<!-- local image -->
![Adorbium cellular automaton gliders](../assets/adorbium_4.gif)

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="Adamata animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    grid_display = grid[0].permute(1,2,0)
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = aa(grid)
    
    grid_display = grid[0].permute(1,2,0)
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()

In [ ]:
# These parameters in AdamAutomaton support adorbium gliders

#print(aa.kernel_diameter, aa.epsilon, aa.alpha, aa.beta_1, aa.beta_2, "mu_g", 0.167, "sigma_g", 0.013, "mu_k", 0.5, "sigma_k", 0.15)
# values
# 27 1e-08 0.1 0.8 0.99 mu_g 0.167 sigma_g 0.013 mu_k 0.5 sigma_k 0.15

class AdamAutomaton():
    
    def __init__(self, **kwargs):
        
        self.kernel_diameter = kwargs["diameter"] if "diameter" in kwargs.keys() else 27
        
        alpha = kwargs["alpha"] if "alpha" in kwargs.keys() else 1e-1
        beta_1 = kwargs["beta_1"] if "beta_1" in kwargs.keys() else 1e-3
        beta_2 = kwargs["beta_2"] if "beta_2" in kwargs.keys() else 1e-4
        epsilon = kwargs["epsilon"] if "epsilon" in kwargs.keys() else 1e-8
        
        self.set_alpha(alpha)
        self.set_beta_1(beta_1)
        self.set_beta_2(beta_2)
        self.set_epsilon(epsilon)
        
        self.init_kernel()
        self.init_growth()
    
    def init_kernel(self, mu=0.5, sigma=0.15):
        
        my_range = np.arange(-1,\
                             1 + 2/(self.kernel_diameter-1),\
                            2/(self.kernel_diameter-1))
        my_range = my_range[:self.kernel_diameter]
        xx, yy = np.meshgrid(my_range, my_range)
        
        
        
        rr = torch.tensor(np.sqrt(xx**2 + yy**2)[None,None,:,:])
        
        self.kernel = torch.exp(-(rr-mu)**2/(2*sigma**2))
        
        self.kernel /= self.kernel.sum()
        self.kernel = self.kernel.to(torch.get_default_dtype())
        
        self.neighborhood = nn.Conv2d(1, 1, \
                self.kernel_diameter,\
                padding=(self.kernel_diameter-1) // 2, \
                groups=1,\
                padding_mode="circular",\
                bias=False)
        
        for param in self.neighborhood.named_parameters():
            param[1].requires_grad = False
            param[1][:] = self.kernel
    
    def init_growth(self, mu=0.167, sigma=0.013):
    
        def growth(x):
            
            return 2*torch.exp(-(x-mu)**2/(2*sigma**2))-1
        
        self.growth = growth
    
    def __call__(self, grid):
        
        # cell states
        a = grid[:,0:1,:,:]
        
        # neighborhoods
        #n = F.conv2d(a, self.kernel)
        n = self.neighborhood(a)
        
        # first and second moments
        m_0 = grid[:,1:2,:,:]
        v_0 = grid[:,2:3,:,:]
        
        # 'gradient', 
        g = self.growth(n)
        
        m = (self.beta_1) * m_0 + (1-self.beta_1)  * g
        v = (self.beta_2) * v_0 + (1-self.beta_2)  * g**2
        
        # adam update for cell states
        new_a = a + self.alpha * (m / (torch.sqrt(v) + self.epsilon))
        
        new_grid = torch.zeros_like(grid)
        # assign cell states and moments
        new_grid[:,0:1,:,:] = new_a.unsqueeze(0).unsqueeze(0)
        new_grid[:,1:2,:,:] = m.unsqueeze(0).unsqueeze(0)
        new_grid[:,2:3,:,:] = v.unsqueeze(0).unsqueeze(0)
        
        return torch.clamp(new_grid, 0, 1.0)
    
    def set_alpha(self, new_alpha):
        self.alpha = 1.0 * new_alpha
        
    def get_alpha(self):
        return 1.0 * self.alpha
        
    def set_beta_1(self, new_beta_1):
        self.beta_1 = 1.0 * new_beta_1
        
    def get_beta_1(self):
        return 1.0 * self.beta_1

    def set_beta_2(self, new_beta_2):
        self.beta_2 = 1.0 * new_beta_2
        
    def get_beta_2(self):
        return 1.0 * self.beta_2
        
    def set_epsilon(self, new_epsilon):
        self.epsilon = 1.0 * new_epsilon
        
    def get_epsilon(self):
        return 1.0 * self.epsilon

# Selecting gliders in adam automaton



In [ ]:
aa = AdamAutomaton(diameter=27, beta_1=0.8, beta_2=0.99)
aa.init_growth(mu=0.167, sigma=0.013)

plt.figure()
plt.imshow(aa.kernel.squeeze(), cmap="magma")
plt.title("Adam Automaton neighborhood kernel")
plt.show()

In [ ]:
my_seed = 13
num_frames = 256
grid_dim = 512

torch.manual_seed(my_seed)

grid = torch.zeros(1,3,grid_dim, grid_dim )
grid[:,:,:,:] = torch.rand(1,3,grid_dim, grid_dim )

gap_size = 96
gap_minus = 10

for ii in range(0, grid_dim, gap_size):
    grid[:,:,ii:ii+gap_size-gap_minus,:] *= 0.0

for jj in range(0, grid_dim, gap_size):
    grid[:,:,:,jj:jj+gap_size-gap_minus] *= 0.0    

fig, ax = plot_grid(grid)
fig.suptitle("Initial conditions: uniform random patches")


IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
grid_0 = grid *1.0

plt.figure(); plt.imshow(grid_0.squeeze().permute(1,2,0)); 
plt.title(f"grid after {num_frames} update steps")
plt.show()

In [ ]:
# select vanguard edges
grid = grid_0 * 1.0

grid[:,0,:100,:] *= 0
grid[:,0,300:,:] *= 0
grid[:,0,:,50:300] *= 0

grid[:,0,100:250,:200] *= 0
grid[:,0,140:,200:] *= 0

plt.figure(); plt.imshow(grid.squeeze().permute(1,2,0)); plt.show()

In [ ]:
num_frames = 384
fig, ax = plot_grid(grid)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
# display with numbered tick marks
plt.figure(); plt.imshow(grid.squeeze().permute(1,2,0)); plt.show()

In [ ]:
# crop gliders
adam_orbium_2 = grid[:,:,10:50, 390:430]
adam_orbium_3 = grid[:,:,355:395, 475:515]

# Display selected gliders
plt.figure();
plt.subplot(121)
plt.imshow(adam_orbium_2.squeeze().permute(1,2,0))
plt.subplot(122)
plt.imshow(adam_orbium_3.squeeze().permute(1,2,0))
plt.suptitle("Selected Adorbium gliders")
plt.show()

In [ ]:
# save patterns if desired
if(0):
    torch.save(adam_orbium_2, os.path.join("..", "patterns", "adam_orbium_2.pt"))
    torch.save(adam_orbium_3, os.path.join("..", "patterns", "adam_orbium_3.pt"))

In [ ]:
if (0):
    adam_orbium_0 = torch.load(os.path.join("..", "patterns", "adam_orbium_0.pt"))
    adam_orbium_1 = torch.load(os.path.join("..", "patterns", "adam_orbium_1.pt"))
    adam_orbium_2 = torch.load(os.path.join("..", "patterns", "adam_orbium_2.pt"))
    adam_orbium_3 = torch.load(os.path.join("..", "patterns", "adam_orbium_3.pt"))

    plt.figure();
    plt.subplot(221)
    plt.imshow(adam_orbium_0.squeeze().permute(1,2,0))
    plt.subplot(222)
    plt.imshow(adam_orbium_1.squeeze().permute(1,2,0))

    plt.subplot(223)
    plt.imshow(adam_orbium_2.squeeze().permute(1,2,0))
    plt.subplot(224)
    plt.imshow(adam_orbium_3.squeeze().permute(1,2,0))
    plt.suptitle("Selected Adorbium gliders")
    plt.show()

    grid = torch.zeros(1,3,96,96)
    grid[:,2,:,:] = adam_orbium_0[:,2,:,:].mean()
    grid[:,:,:adam_orbium_0.shape[-2], :adam_orbium_0.shape[-1]] = adam_orbium_0

    grid[:,:,:adam_orbium_1.shape[-2],\
         -adam_orbium_1.shape[-1]:] = adam_orbium_1
    grid[:,:, -adam_orbium_2.shape[-1]:,\
         :adam_orbium_2.shape[-2]] = adam_orbium_2

    grid[:,:, -adam_orbium_3.shape[-2]:, :adam_orbium_3.shape[-1]] = adam_orbium_3
    
    num_frames = 384
    fig, ax = plot_grid(grid)
    plt.show()

    matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).save(os.path.join("..","assets", "adorbium_4.gif"))


# Grid search discretization experiment

In [ ]:
adam_orbium_1 = torch.load(os.path.join("..","patterns","adam_orbium_1.pt"))

plt.imshow(adam_orbium_1.squeeze().permute(1,2,0))

In [ ]:
save_figs = False
full_run = False

if full_run:
    # to run a full grid search experiment (takes a while)
    grid_dim = 180
    number_steps = 1024
    warmup_steps = number_steps // 2

    my_dts = np.arange(0.01, 1.02, 0.04) 
    my_krs = np.arange(9,32,1)
    native_diameter = 27
else:
    # smaller experimental run, for getting started
    grid_dim = 180
    number_steps = 102
    warmup_steps = number_steps // 2


    my_dts = np.arange(0.01, 1.02, 0.1) 
    my_krs = np.arange(9,33,6)
    native_diameter = 27

In [ ]:
stability_map = None
for my_kr in my_krs:
    my_diameter = 2 * my_kr + 1
    temp_map = None
    
    for my_dt in my_dts:
        
        aa = AdamAutomaton(diameter=my_diameter, beta_1=0.8, beta_2=0.99)
        aa.init_growth(mu=0.167, sigma=0.013)
        
        aa.set_alpha(my_dt)
        
        
        scale = my_diameter / native_diameter
        adam_orbium_1 = np.array(torch.load(os.path.join("..","patterns","adam_orbium_1.pt"))).squeeze()
        
        scale_dims = [np.round(elem * scale) for elem in adam_orbium_1.shape[-2:]]
        adam_orbium_scaled = skimage.transform.resize(adam_orbium_1.transpose(1,2,0), scale_dims)
        adam_orbium_scaled = torch.tensor(adam_orbium_scaled).permute(2,0,1).unsqueeze(0)

        grid = torch.zeros(1,3,grid_dim, grid_dim)
        grid[:,-1,:,:] = adam_orbium_scaled[:,-1,:,:].mean()
        grid[:,:,:adam_orbium_scaled.shape[-2], :adam_orbium_scaled.shape[-1]] = adam_orbium_scaled

        for my_step in range(warmup_steps):
            grid = aa(grid)

        grid_0 = 1.0 * grid
        
        for my_step in range(number_steps):
            grid = aa(grid)

        fig, ax = plt.subplots(1,2, figsize=(4.5,3.5), facecolor="white")

        grid_display = grid[0].permute(1,2,0)
        grid_0_display = grid_0[0].permute(1,2,0)


        ax[0].imshow(grid_0_display, interpolation="nearest")
        ax[0].set_title("Starting grid")
        ax[1].imshow(grid_display, interpolation="nearest")

        ax[1].set_title(f"step {my_step+1}\n dt: {aa.get_alpha():.4f} \n kernel diameter: {aa.kernel.shape[-1]}")

        ax[0].set_yticklabels('')
        ax[0].set_xticklabels('')
        ax[1].set_yticklabels('')
        ax[1].set_xticklabels('')

        fig.suptitle("Adam Orbium discretization stability")

        fig_name = f"step{my_step+1}_dt{aa.get_alpha():.4f}_kd{aa.kernel.shape[-1]}"
        fig_name =fig_name.replace(".","x") + ".png"
        dir_name = os.path.join("..","assets", "adamata_stability")
        if os.path.isdir(dir_name):
            pass
        else:
            os.mkdir(dir_name)

        fig_name = os.path.join(dir_name, fig_name)
        if save_figs:
            plt.savefig(fig_name)
        
        if temp_map is None:
            temp_map = grid_display
            
        else:
            temp_map = np.append(temp_map, grid_display, axis=1)

        plt.show()
    
    if stability_map is None:
        stability_map = temp_map
    else:
        stability_map = np.append(stability_map, temp_map, axis=0)
        
    stability_figure_name = os.path.join(dir_name, "stability_map.png")
    stability_numpy_name = os.path.join(dir_name, "stability_map.npy")
    
    if save_figs:
        sio.imsave(stability_figure_name, stability_map)
        np.save(stability_numpy_name, stability_map)
